In [1]:
# data
import nfl_data_py as nfl

# data loading and plotting
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# pipeline
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.feature_selection import RFECV, RFE
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, nan_euclidean_distances

df_ids = nfl.import_ids()
df_weekly = nfl.import_weekly_data(range(2018, 2025))
df_seasonal = nfl.import_seasonal_data(range(2018,2025))
df_schedule = nfl.import_schedules(range(2018, 2025))
df_pass_pfr = nfl.import_weekly_pfr('pass', range(2018, 2025))
df_rush_pfr = nfl.import_weekly_pfr('rush', range(2018, 2025))
df_rec_pfr = nfl.import_weekly_pfr('rec', range(2018, 2025))
df_pass_ngs = nfl.import_ngs_data('passing',range(2018, 2025))
df_rush_ngs = nfl.import_ngs_data('rushing',range(2018, 2025))
df_rec_ngs = nfl.import_ngs_data('receiving',range(2018, 2025))
df_snap_counts = nfl.import_snap_counts(range(2018, 2025))


# 
df_weekly = df_weekly[(df_weekly['season_type'] == 'REG') & (df_weekly['position'].isin(['QB', 'WR', 'RB', 'TE']))]

# 
df_weekly['game_id_home_away'] = df_weekly['season'].astype(str) + '_' + df_weekly['week'].apply(lambda x: f"{x:02d}")+'_'+df_weekly['recent_team']+'_'+df_weekly['opponent_team']
df_weekly['game_id_away_home'] = df_weekly['season'].astype(str) + '_' + df_weekly['week'].apply(lambda x: f"{x:02d}")+'_'+df_weekly['opponent_team']+'_'+df_weekly['recent_team']

# Rename player_id's to later merge using the consistent player_id
df_ids = df_ids.rename(columns={'gsis_id': 'player_id', 'pfr_id': 'pfr_player_id'})
df_pass_ngs = df_pass_ngs.rename(columns={'player_gsis_id': 'player_id'})
df_rush_ngs = df_rush_ngs.rename(columns={'player_gsis_id': 'player_id'})
df_rec_ngs = df_rec_ngs.rename(columns={'player_gsis_id': 'player_id'})

df_merged = pd.melt(
    df_weekly,
    id_vars=['player_id', 'position', 'season', 'week', 'recent_team', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'passing_2pt_conversions', 'interceptions', 'sack_fumbles_lost', 'sacks', 'sack_yards', 'passing_air_yards', 'passing_epa', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_2pt_conversions', 'rushing_fumbles_lost', 'rushing_epa', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_2pt_conversions', 'receiving_fumbles_lost', 'racr', 'wopr', 'receiving_epa', 'fantasy_points'],
    value_vars=['game_id_home_away', 'game_id_away_home'],
    var_name='game_id_type',
    value_name='game_id'
)


df_merged = pd.merge(df_merged, df_ids[['player_id', 'pfr_player_id', 'draft_pick', 'draft_year']], on = 'player_id', how = 'inner')
df_merged = pd.merge(df_merged, df_seasonal[['player_id', 'season', 'dom']], on = ['player_id', 'season'], how = 'left')
df_merged = pd.merge(df_merged, df_schedule[['game_id', 'home_team']], on='game_id', how='inner')
df_merged = pd.merge(df_merged, df_pass_pfr[['pfr_player_id', 'season', 'week', 'passing_bad_throws', 'times_pressured']], on = ['pfr_player_id', 'season', 'week'], how = 'left')
df_merged = pd.merge(df_merged, df_rec_pfr[['pfr_player_id', 'season', 'week', 'receiving_rat']], on = ['pfr_player_id', 'season', 'week'], how = 'left')
df_merged = pd.merge(df_merged, df_rush_pfr[['pfr_player_id', 'season', 'week', 'rushing_broken_tackles']], on = ['pfr_player_id', 'season', 'week'], how = 'left')
df_merged = pd.merge(df_merged, df_pass_ngs[['player_id', 'season', 'week', 'passer_rating', 'aggressiveness']], on = ['player_id', 'season', 'week'], how = 'left')
df_merged = pd.merge(df_merged, df_rec_ngs[['player_id', 'season', 'week', 'catch_percentage']], on = ['player_id', 'season', 'week'], how = 'left')
df_merged = pd.merge(df_merged, df_rush_ngs[['player_id', 'season', 'week', 'efficiency']], on = ['player_id', 'season', 'week'], how = 'left')
df_merged = pd.merge(df_merged, df_snap_counts[['pfr_player_id', 'season', 'week', 'offense_snaps']], on = ['pfr_player_id', 'season', 'week'], how = 'left')

df_merged = df_merged.drop(columns=['game_id', 'game_id_type', 'pfr_player_id'])


df_merged = df_merged.fillna(0)

df_merged['rookie_flag'] = (df_merged['season'] == df_merged['draft_year']).astype(int)
df_merged['last_season_data_flag'] = (df_merged['week'] < 6).astype(int)

recent_team_means = df_merged.groupby(['recent_team', 'season', 'week'])['fantasy_points'].mean().reset_index()
opponent_team_means = df_merged.groupby(['opponent_team', 'season', 'week'])['fantasy_points'].mean().reset_index()
position_means = df_merged.groupby(['position', 'season', 'week'])['fantasy_points'].mean().reset_index()

recent_team_means.rename(columns={'fantasy_points': 'recent_team_encoded'}, inplace=True)
opponent_team_means.rename(columns={'fantasy_points': 'opponent_team_encoded'}, inplace=True)
position_means.rename(columns={'fantasy_points': 'position_encoded'}, inplace=True)

df_merged = pd.merge(df_merged, recent_team_means, on=['recent_team', 'season', 'week'], how='left')
df_merged = pd.merge(df_merged, opponent_team_means, on=['opponent_team', 'season', 'week'], how='left')
df_merged = pd.merge(df_merged, position_means, on=['position', 'season', 'week'], how='left')


df_merged['turnover'] = (
    df_merged['interceptions'] +
    df_merged['sack_fumbles_lost'] +
    df_merged['rushing_fumbles_lost'] +
    df_merged['receiving_fumbles_lost']
)

# calculating fantasy points scored by category
df_merged['rushing_pts'] = (df_merged['rushing_tds'] * 6) + (df_merged['rushing_2pt_conversions'] * 2)
df_merged['receiving_pts'] = (df_merged['receiving_tds'] * 6) + (df_merged['receiving_2pt_conversions'] * 2)
df_merged['passing_pts'] = (df_merged['passing_tds'] * 6) + (df_merged['passing_2pt_conversions'] * 2)

df_merged['epa_total'] = df_merged['passing_epa'] + df_merged['rushing_epa'] + df_merged['receiving_epa']

# Dropping features that were integrated into other features above
df_merged = df_merged.drop(columns=['draft_year', 'interceptions', 'sack_fumbles_lost', 'rushing_fumbles_lost', 'receiving_fumbles_lost', 
                                    'rushing_tds', 'rushing_2pt_conversions', 'receiving_tds', 'receiving_2pt_conversions', 'passing_tds', 
                                    'passing_2pt_conversions', 'passing_epa', 'rushing_epa', 'receiving_epa'])

# features for whom rolling features should be created
columns_to_roll = ['completions', 'attempts', 'passing_yards', 'sacks', 'passer_rating', 'aggressiveness', 'catch_percentage', 'efficiency',
                   'sack_yards', 'passing_air_yards', 'pacr', 'carries', 'offense_snaps', 'recent_team_encoded', 'opponent_team_encoded', 
                   'rushing_yards', 'receptions', 'targets', 'receiving_yards', 'racr', 'wopr', 'fantasy_points', 'passing_bad_throws', 
                   'times_pressured', 'position_encoded', 'epa_total', 'receiving_rat', 'rushing_broken_tackles', 'turnover', 'rushing_pts', 
                   'receiving_pts', 'passing_pts']


def create_rolling_features(df, columns_to_roll):
    # Sort dataframe
    df = df.sort_values(['player_id', 'season', 'week']).reset_index(drop=True)
    
    # Create empty dictionary to store all new features
    new_features = {}
    
    for col in columns_to_roll:
        # 5-week window stats
        grouped = df.groupby('player_id')[col].shift(1)
        
        new_features[f'ewm_{col}_l5w'] = (grouped
            .ewm(span=5, min_periods=5)
            .mean())
        
        for metric in ['mean', 'median', 'std']:
            new_features[f'{metric}_{col}_l5w'] = (grouped
                .rolling(window=5, min_periods=5)
                .agg(metric))
        
        # 3-week window stats
        for metric in ['max', 'min']:
            new_features[f'{metric}_{col}_l3w'] = (grouped
                .rolling(window=3, min_periods=3)
                .agg(metric))
    
    # Add fantasy points feature
    new_features['cnt_games_over_20ffpts_l5w'] = (
        df.groupby('player_id')['fantasy_points']
        .shift(1)
        .rolling(window=5, min_periods=5)
        .apply(lambda x: (x > 20).sum())
    )
    
    # Combine all new features at once
    feature_df = pd.DataFrame(new_features)
    
    # Combine with original dataframe
    return pd.concat([df, feature_df], axis=1)


# Anwendung auf df_merged
df_merged = create_rolling_features(df_merged, columns_to_roll)


df_merged = df_merged.dropna()

df_merged = df_merged.drop(columns=['completions', 'attempts', 'passing_yards', 'sacks', 'sack_yards', 'passing_air_yards',  
                                    'pacr', 'carries', 'rushing_yards', 'receptions', 'targets', 'receiving_yards', 'racr', 
                                    'wopr', 'passing_bad_throws', 'times_pressured', 'receiving_rat', 'rushing_broken_tackles', 'turnover', 
                                    'rushing_pts', 'receiving_pts', 'passing_pts', 'home_team', 'passer_rating', 'aggressiveness',
                                    'catch_percentage', 'efficiency', 'offense_snaps', 'recent_team_encoded', 'opponent_team_encoded', 
                                    'position_encoded', 'recent_team', 'opponent_team', 'position', 'epa_total'])


df_merged = df_merged.sort_values(['player_id', 'season', 'week'])
df_merged['did_play'] = 1

player_seasons = df_merged[['player_id', 'season']].drop_duplicates()

all_weeks = []

for _, row in player_seasons.iterrows():
    # Assuming weeks go from 1 to 18 for NFL season
    weeks = pd.DataFrame({
        'player_id': row['player_id'],
        'season': row['season'],
        'week': range(1, 19),
    })
    all_weeks.append(weeks)
    
complete_weeks = pd.concat(all_weeks, ignore_index=True)

df_merged = pd.merge(
    complete_weeks,
    df_merged,
    on=['player_id', 'season', 'week'],
    how='left'
)

df_merged = df_merged.fillna(0)


df_seq = df_merged.copy()

df_seq['time_index'] = df_seq['season'] * 100 + df_seq['week']

df_seq = df_seq.sort_values(['player_id', 'time_index'])


df_predict = df_seq[df_seq['time_index'] > 202318]
df_train = df_seq[df_seq['time_index'] < 202401]

Downcasting floats.


In [3]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lstm_data_loader import load_data, edit_data, create_train_data, prepare_data, create_sequences, create_model
from tensorflow.keras.callbacks import EarlyStopping


# Lade die Trainingsdaten
df = load_data()
df = edit_data(df)
df_train_2 = create_train_data(df)

KeyboardInterrupt: 